In [1]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘caret’ was built under R version 4.2.3”
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked fro

In [2]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100,500,1000)
time_off<-c(10000,10,20,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [3]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [4]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()

corpus_list<-c("coha")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("UnTagged")

ppmi_setting_list<-c("PPMI")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [5]:
for (c in corpus_list){
  for (t in tagged_list) {
    for (f in feature_setting_list) {
      for (p in ppmi_setting_list){
        for (a in comp_setting_list){
          for (i in time_off) {
            
            for (j in cut_off) {
              
              
              print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
              
              na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
              nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
              
              for (r in ratings_list){
                na_df<-na_df %>% filter(source==r)
                nona_df<-nona_df %>% filter(source==r)
                
                trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                trainY_na<-na_df %>% select(compositionality)
                trainY_na<-trainY_na$compositionality
                
                
                trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                trainY_nona<-nona_df %>% select(compositionality)
                trainY_nona<-trainY_nona$compositionality
                
                if (dim(trainX_nona)[1]<10) {
                  break
                }
                for (s in seed_list)  {
                  set.seed(s)
                  seeds <- vector(mode = "list", length = 14)
                  for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                  #for the last model
                  seeds[[14]]<-sample.int(1000, 1)
                  
                  ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                  )
                  
                  elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                  )
                  
                  lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                  )
                  
                  
                  ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  
                  ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                  )
                  
                  elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                  )
                  
                  lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                  )
                  
                  
                  ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  
                  perf_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                  perf_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                  perf_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                  
                  
                  perf_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                  perf_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                  perf_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                  
                  varimp_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_na)$importance))
                  varimp_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_na)$importance))
                  varimp_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_na)$importance))
                  
                  varimp_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_nona)$importance))
                  varimp_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_nona)$importance))
                  varimp_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_nona)$importance))    
                  
                  
                  
                  list_of_rsqr[[m]]<-perf_elastic_na
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_elastic_nona
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_ridge_na
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_ridge_nona
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_lasso_na
                  m<-m+1 
                  
                  list_of_rsqr[[m]]<-perf_lasso_nona
                  m<-m+1 
                  
                  if (i==10000) {
                    list_of_vi_10000[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_lasso_nona
                    v<-v+1
                    
                  }
                  else if (i==10) {
                    list_of_vi_10[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  
                  else if (i==20) {
                    list_of_vi_20[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  else if (i==50) {
                    list_of_vi_50[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  else if (i==100) {
                    list_of_vi_100[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  
                }
                
              }
            }
          }
        }
      }
    }
  }
}

[1] "coha PPMI Aware withSetting UnTagged 10000 0"
[1] "coha PPMI Aware withSetting UnTagged 10000 10"
[1] "coha PPMI Aware withSetting UnTagged 10000 50"
[1] "coha PPMI Aware withSetting UnTagged 10000 100"
[1] "coha PPMI Aware withSetting UnTagged 10000 500"
[1] "coha PPMI Aware withSetting UnTagged 10000 1000"
[1] "coha PPMI Aware withSetting UnTagged 10 0"
[1] "coha PPMI Aware withSetting UnTagged 10 10"
[1] "coha PPMI Aware withSetting UnTagged 10 50"
[1] "coha PPMI Aware withSetting UnTagged 10 100"
[1] "coha PPMI Aware withSetting UnTagged 10 500"
[1] "coha PPMI Aware withSetting UnTagged 10 1000"
[1] "coha PPMI Aware withSetting UnTagged 20 0"
[1] "coha PPMI Aware withSetting UnTagged 20 10"
[1] "coha PPMI Aware withSetting UnTagged 20 50"
[1] "coha PPMI Aware withSetting UnTagged 20 100"
[1] "coha PPMI Aware withSetting UnTagged 20 500"
[1] "coha PPMI Aware withSetting UnTagged 20 1000"
[1] "coha PPMI Aware withSetting UnTagged 50 0"
[1] "coha PPMI Aware withSetting UnTagged 5

In [10]:
google_df <-read.csv(paste0("/data/dharp/compounds/datasets/coha/features_CompoundAware_withSetting_PPMI_UnTagged_10_0_med.csv"),sep = '\t')

#google_df$compound_label<-cut_number(google_df$compositionality,n=3,labels=c("low","med","high"))
#google_df$corpus<-"google"

#combined_df<-rbind(coha_df,google_df)

#combined_df %>% filter(source=="reddy") %>%ggplot(aes(fill=compound_label,x=comp_tf.0)) + facet_grid(~corpus)+geom_density(alpha = 0.3) + theme_bw()

reddy_df<-google_df %>% filter(source=="reddy")

#cordeiro90_df<-google_df %>% filter(source=="cordeiro90")
#cordeiro100_df<-google_df %>% filter(source=="cordeiro100")

trainX<-reddy_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
trainY<-reddy_df %>% select(compositionality)
trainY<-trainY$compositionality


for (s in seed_list)  {
  set.seed(s)
  seeds <- vector(mode = "list", length = 14)
  for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
  #for the last model
  seeds[[14]]<-sample.int(1000, 1)

  ridge_nona_coha <- train(trainX,trainY,method = "glmnet",metric = "Rsquared",
                           trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                           preProc = c("nzv","center", "scale")
  )
  print(getTrainPerf(ridge_nona_coha)[,"TrainRsquared"])

}

[1] 0.5147677
[1] 0.4472353
[1] 0.4870363
[1] 0.5061195
[1] 0.5017567
[1] 0.4814031
[1] 0.4848378
[1] 0.531367
[1] 0.491554
[1] 0.4797273


In [27]:
varImp_df$names <- rownames(varImp_df)

In [28]:
varImp_df[order(-varImp_df$Overall), ]

,Overall,names
,<dbl>,<chr>
comp_tf.2010,100.00000,comp_tf.2010
log_comp_freq.2010,99.98601,log_comp_freq.2010
comp_tf.2000,96.91917,comp_tf.2000
log_comp_freq.2000,96.89357,log_comp_freq.2000
comp_freq.2010,92.27417,comp_freq.2010
comp_freq.2000,86.45822,comp_freq.2000
log_comp_freq.1980,83.94510,log_comp_freq.1980
comp_tf.1980,83.86297,comp_tf.1980
local_mi.2010,81.81439,local_mi.2010


In [42]:
varimp_df<-bind_rows(list_of_vi_10000)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0
vi_statistics<-varimp_df %>% group_by(timespan,cutoff) %>% 
summarise_all(mean)
vi_statistics

timespan,cutoff,feature,impute,tag,setting,dataset,corpus,ml_algo,n,⋯,sim_bw_settings_head.0,sim_bw_settings_modifier.0,sim_cpf_0.0,sim_cpf_100.0,sim_cpf_25.0,sim_cpf_50.0,sim_cpf_75.0,sim_cpf_beta.0,sim_with_head.0,sim_with_modifier.0
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10000,0,NA,NA,NA,NA,NA,NA,NA,91.0,⋯,45.73981,81.44568,8.492177,26.276171,2.776119,6.191089,1.967925,9.336969,8.791513,47.56116
10000,10,NA,NA,NA,NA,NA,NA,NA,91.0,⋯,51.66532,59.74900,9.576280,12.129149,5.129055,5.381084,5.219122,24.366519,9.177394,32.86125
10000,50,NA,NA,NA,NA,NA,NA,NA,91.0,⋯,71.61407,63.78103,9.213907,3.829191,6.048057,4.445103,4.290913,14.128180,8.050580,20.23224
10000,100,NA,NA,NA,NA,NA,NA,NA,91.0,⋯,83.96942,70.40950,36.048909,7.620920,15.521747,13.431156,11.332139,12.684058,4.199695,17.64713
10000,500,NA,NA,NA,NA,NA,NA,NA,90.5,⋯,54.83081,50.03155,3.058252,6.260842,4.971937,3.936224,1.150443,3.088325,24.189562,46.92141
10000,1000,NA,NA,NA,NA,NA,NA,NA,89.0,⋯,33.52256,44.20439,2.130765,2.912598,3.638295,2.721439,2.976852,3.005555,27.020310,35.38870


In [27]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
statistics<-rsquared_df %>% group_by(impute,setting,dataset,ml_algo,timespan,cutoff) %>% 
summarise(n=mean(n),meanRsquared=mean(TrainRsquared),sdRsquared=sd(TrainRsquared),meanCorr=mean(TrainSpearman),sdCorr=sd(TrainSpearman))

statistics %>% arrange(desc(meanCorr)) %>% head()

`summarise()` has grouped output by 'impute', 'setting', 'dataset', 'ml_algo', 'timespan'. You can override using the `.groups` argument.


impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
r,Aware,reddy,elastic,10,1000,66,0.5602920,0.04296199,0.7792180,0.03022483
r,Aware,reddy,lasso,10,1000,66,0.5563630,0.04459371,0.7713386,0.03197927
python,Aware,reddy,elastic,10,1000,91,0.5313802,0.03933757,0.7347352,0.02557355
python,Aware,reddy,rigde,10,1000,91,0.4924387,0.02935631,0.7246680,0.02214134
python,Aware,reddy,lasso,10,1000,91,0.5310502,0.03957151,0.7230866,0.03434580
r,Aware,reddy,elastic,10,0,91,0.5306651,0.05789085,0.7176905,0.10400215


In [29]:
statistics %>% filter(dataset=="reddy" & n==91) %>% arrange(desc(meanCorr)) %>% head()

impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
python,Aware,reddy,elastic,10,1000,91,0.5313802,0.03933757,0.7347352,0.02557355
python,Aware,reddy,rigde,10,1000,91,0.4924387,0.02935631,0.7246680,0.02214134
python,Aware,reddy,lasso,10,1000,91,0.5310502,0.03957151,0.7230866,0.03434580
r,Aware,reddy,elastic,10,0,91,0.5306651,0.05789085,0.7176905,0.10400215
r,Aware,reddy,rigde,10,0,91,0.5183447,0.06577922,0.7140884,0.10558405
python,Aware,reddy,elastic,10,500,91,0.4878255,0.02375583,0.7016591,0.02428786


In [32]:
statistics %>% filter(dataset=="cordeiro90" ) %>% arrange(desc(meanCorr)) %>% head()

impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [155]:
write.csv(rsquared_df,"rsquared_Tagged_PPMI_Aware.csv",row.names = FALSE)

In [156]:
write.csv(varimp_df,"varimp_Tagged_PPMI_Aware.csv",row.names = FALSE)